In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 📊 Interactive Visualizations\n",
    "### Building dashboard components with Plotly\n",
    "\n",
    "This notebook creates:\n",
    "- Multi-year flood timing overlays\n",
    "- Flash flood analysis (top 1% events)\n",
    "- Interactive time series plots\n",
    "- Risk assessment charts"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import plotly.graph_objects as go\n",
    "from plotly.subplots import make_subplots\n",
    "from src.config import Config\n",
    "from src.data_loader import ERA5DataLoader\n",
    "from src.analytics import FloodAnalytics"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "station_key = 'koshi_chatara'\n",
    "station_config = Config.TARGET_STATIONS[station_key]\n",
    "\n",
    "loader = ERA5DataLoader(station_config, years_back=5)\n",
    "data = loader.fetch_era5_data()\n",
    "\n",
    "print(f\"Station: {station_config['name']}\")\n",
    "print(f\"Years: {sorted(data['year'].unique())}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Flood Timing Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Multi-year overlay showing seasonal patterns\n",
    "years = sorted(data['year'].unique())\n",
    "colors = ['#dc143c', '#003893', '#f59e0b', '#10b981', '#8b5cf6']\n",
    "\n",
    "fig = go.Figure()\n",
    "\n",
    "for idx, year in enumerate(years):\n",
    "    year_data = data[data['year'] == year].sort_values('day_of_year')\n",
    "    \n",
    "    fig.add_trace(go.Scatter(\n",
    "        x=year_data['day_of_year'],\n",
    "        y=year_data['water_level_m'],\n",
    "        mode='lines',\n",
    "        name=str(year),\n",
    "        line=dict(color=colors[idx % len(colors)], width=1.5)\n",
    "    ))\n",
    "\n",
    "# Add monsoon season shading\n",
    "fig.add_vrect(x0=152, x1=273, fillcolor=\"lightblue\", opacity=0.1, \n",
    "              layer=\"below\", line_width=0)\n",
    "\n",
    "fig.update_layout(\n",
    "    title=\"Multi-Year Flood Timing Comparison\",\n",
    "    xaxis_title=\"Day of Year\",\n",
    "    yaxis_title=\"Water Level (m)\",\n",
    "    height=500,\n",
    "    hovermode='x unified'\n",
    ")\n",
    "\n",
    "fig.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Flash Flood Analysis (Top 1%)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Identify extreme events\n",
    "fig = go.Figure()\n",
    "\n",
    "for idx, year in enumerate(years):\n",
    "    year_data = data[data['year'] == year]\n",
    "    \n",
    "    # Top 1% threshold\n",
    "    threshold_99 = year_data['water_level_m'].quantile(0.99)\n",
    "    extremes = year_data[year_data['water_level_m'] >= threshold_99]\n",
    "    \n",
    "    fig.add_trace(go.Scatter(\n",
    "        x=extremes['day_of_year'],\n",
    "        y=extremes['water_level_m'],\n",
    "        mode='markers',\n",
    "        name=str(year),\n",
    "        marker=dict(size=8, color=colors[idx % len(colors)], opacity=0.7)\n",
    "    ))\n",
    "\n",
    "# Add flood stage line\n",
    "fig.add_hline(y=station_config['flood_stage'], line_dash=\"dash\", \n",
    "              line_color=\"orange\", annotation_text=\"Flood Stage\")\n",
    "\n",
    "fig.update_layout(\n",
    "    title=\"Flash Flood Analysis - Top 1% Extreme Events\",\n",
    "    xaxis_title=\"Day of Year\",\n",
    "    yaxis_title=\"Water Level (m)\",\n",
    "    height=500\n",
    ")\n",
    "\n",
    "fig.show()\n",
    "\n",
    "# Summary statistics\n",
    "print(\"\\nExtreme Events Summary:\")\n",
    "for year in years:\n",
    "    year_data = data[data['year'] == year]\n",
    "    threshold_99 = year_data['water_level_m'].quantile(0.99)\n",
    "    extremes = year_data[year_data['water_level_m'] >= threshold_99]\n",
    "    print(f\"{year}: {len(extremes)} extreme events, threshold={threshold_99:.2f}m\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Seasonal Risk Heatmap"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate average water level by month and year\n",
    "pivot = data.pivot_table(\n",
    "    values='water_level_m',\n",
    "    index='month',\n",
    "    columns='year',\n",
    "    aggfunc='mean'\n",
    ")\n",
    "\n",
    "fig = go.Figure(data=go.Heatmap(\n",
    "    z=pivot.values,\n",
    "    x=pivot.columns,\n",
    "    y=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', \n",
    "       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],\n",
    "    colorscale='RdYlBu_r',\n",
    "    colorbar=dict(title=\"Water Level (m)\")\n",
    "))\n",
    "\n",
    "fig.update_layout(\n",
    "    title=\"Monthly Average Water Levels Across Years\",\n",
    "    xaxis_title=\"Year\",\n",
    "    yaxis_title=\"Month\",\n",
    "    height=500\n",
    ")\n",
    "\n",
    "fig.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Discharge Distribution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Box plot by season\n",
    "fig = go.Figure()\n",
    "\n",
    "seasons = ['Winter', 'Pre-Monsoon', 'Monsoon', 'Post-Monsoon']\n",
    "for season in seasons:\n",
    "    season_data = data[data['season'] == season]['discharge_cumecs']\n",
    "    fig.add_trace(go.Box(y=season_data, name=season))\n",
    "\n",
    "fig.update_layout(\n",
    "    title=\"Discharge Distribution by Season\",\n",
    "    yaxis_title=\"Discharge (m³/s)\",\n",
    "    height=500,\n",
    "    showlegend=False\n",
    ")\n",
    "\n",
    "fig.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ Visualization Insights\n",
    "\n",
    "- Interactive Plotly charts enable deep data exploration\n",
    "- Multi-year overlays reveal consistent seasonal patterns\n",
    "- Flash flood analysis identifies timing of extreme events\n",
    "- Heatmaps show monthly variations across years\n",
    "- All visualizations integrated into Streamlit dashboard"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}